In [113]:
'''BASIC'''
import numpy as np
from numpy.linalg import svd, eig
import pandas as pd
import random as rand
import databricks.koalas as ks
import math
import os
import itertools as itr
import warnings
# warnings.simplefilter('ignore')
'''PLOT'''
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import cm
plt.style.use('seaborn-pastel')
font = {'size':16}
'''STATS'''
import scipy.stats as scs
# from scipy.optimize import nnls
# import statsmodels.api as sm
# '''SCIKIT LEARN'''
# # from sklearn.datasets import load_boston
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV
# from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
# from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostRegressor, AdaBoostClassifier
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans, AgglomerativeClustering
# from sklearn.metrics import silhouette_score
# '''SRC'''
# from src.cluster_generators import make_gaussian_clusters
# from src.multihist import multihist
# from src.silhouette import make_silhouette_plot
# '''DEEP LEARNING’'''
# import tensorflow as tf
# from tensorflow.keras import layers
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
from sklearn import metrics
from sklearn.metrics import average_precision_score
import pickle
from sklearn.metrics import f1_score
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [8]:
df=pd.read_json("data/data.json")

In [10]:
pandas_profiling.ProfileReport(df)

Number of variables,44
Number of observations,14337
Total Missing (%),1.8%
Total size in memory,4.8 MiB
Average record size in memory,352.0 B
Numeric,18
Categorical,15
Boolean,4
Date,0
Text (Unique),0
Rejected,5


In [19]:
df["name"][1]

'Winthrop RUF Winter Getaway'

In [10]:
# Convert unix date to datetime
from datetime import datetime
def convert_date():
    df["approx_payout_date"] = pd.to_datetime(df["approx_payout_date"], unit = 's')
    df["event_created"] = pd.to_datetime(df["event_created"], unit = 's')
    df["event_end"] = pd.to_datetime(df["event_end"], unit = 's')
    df["event_start"] = pd.to_datetime(df["event_start"], unit = 's')
    return df

In [20]:
df["acct_type"].unique()

array(['fraudster_event', 'premium', 'spammer_warn', 'fraudster',
       'spammer_limited', 'spammer_noinvite', 'locked', 'tos_lock',
       'tos_warn', 'fraudster_att', 'spammer_web', 'spammer'],
      dtype=object)

In [21]:
def create_target():
    df['fraud'] = df['acct_type'].map({'fraudster_event': 1, 
                                   'premium': 0, 
                                   'spammer_warn': 0, 
                                   'fraudster': 1,
                                   'spammer_limited': 0, 
                                   'spammer_noinvite': 0,
                                   'locked': 1, 
                                   'tos_lock': 0,
                                   'tos_warn': 0,
                                   'fraudster_att': 1,
                                   'spammer_web': 0,
                                   'spammer': 0})
    return df

In [33]:
X = df[["user_type", 
        "user_age", 
        "channels", 
        "name_length"]]

In [34]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14337 entries, 0 to 14336
Data columns (total 4 columns):
user_type      14337 non-null int64
user_age       14337 non-null int64
channels       14337 non-null int64
name_length    14337 non-null int64
dtypes: int64(4)
memory usage: 448.1 KB


In [35]:
y = df["fraud"]

In [36]:
X.shape, y.shape

((14337, 4), (14337,))

In [37]:
#Testing our baseline model
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,
                                                        random_state=4)

In [57]:
'''Random Forest Decision Tree'''
def baseline(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=10,
                                criterion='gini',
                                max_depth=None,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                min_weight_fraction_leaf=0.0,
                                max_features='auto',
                                max_leaf_nodes=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None,
                                bootstrap=True,
                                oob_score=False,
                                n_jobs=None,
                                random_state=None,
                                verbose=0,
                                warm_start=False,
                                class_weight=None)
    rf.fit(X_train, y_train)
    importances = rf.feature_importances_
    RF_pred = rf.predict(X_test)
    
#     pickle_out = open("models/baseline_event_model.pkl","wb")
#     pickle.dump(rf, pickle_out)
#     pickle_out.close()
    return "importances:", importances, "precision:", average_precision_score(y_test, RF_pred), "F1:", f1_score(y_test, RF_pred) 

In [62]:
rf.classes_

array([0, 1])

In [60]:
predict_proba = rf.predict_proba(X_test)

In [76]:
predict_proba.shape

(4302, 2)

In [90]:
predict_proba[:,1:2]

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

## Log
First run: with vars df[["user_type", "user_age", "channels", "name_length"]
('importances:',
 array([0.06866372, 0.38180023, 0.14813358, 0.40140247]),
 'precision:',
 0.2625649709914498,
 'F1:',
 0.44692737430167595)

In [58]:
baseline(X_train, X_test, y_train, y_test)

('importances:',
 array([0.06866372, 0.38180023, 0.14813358, 0.40140247]),
 'precision:',
 0.2625649709914498,
 'F1:',
 0.44692737430167595)

In [45]:
#output model to pickle

pickle_out = open("models/baseline_event_model.pkl","wb")
pickle.dump(rf, pickle_out)
pickle_out.close()

# test different models with topic features

In [91]:
topic_df = pd.read_pickle('data/description_topics_df.pickle')

In [92]:
topic_df.head() #400 topics

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24,topic_25,topic_26,topic_27,topic_28,topic_29,topic_30,topic_31,topic_32,topic_33,topic_34,topic_35,topic_36,topic_37,topic_38,topic_39,topic_40,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50,topic_51,topic_52,topic_53,topic_54,topic_55,topic_56,topic_57,topic_58,topic_59,topic_60,topic_61,topic_62,topic_63,topic_64,topic_65,topic_66,topic_67,topic_68,topic_69,topic_70,topic_71,topic_72,topic_73,topic_74,topic_75,topic_76,topic_77,topic_78,topic_79,topic_80,topic_81,topic_82,topic_83,topic_84,topic_85,topic_86,topic_87,topic_88,topic_89,topic_90,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100,topic_101,topic_102,topic_103,topic_104,topic_105,topic_106,topic_107,topic_108,topic_109,topic_110,topic_111,topic_112,topic_113,topic_114,topic_115,topic_116,topic_117,topic_118,topic_119,topic_120,topic_121,topic_122,topic_123,topic_124,topic_125,topic_126,topic_127,topic_128,topic_129,topic_130,topic_131,topic_132,topic_133,topic_134,topic_135,topic_136,topic_137,topic_138,topic_139,topic_140,topic_141,topic_142,topic_143,topic_144,topic_145,topic_146,topic_147,topic_148,topic_149,topic_150,topic_151,topic_152,topic_153,topic_154,topic_155,topic_156,topic_157,topic_158,topic_159,topic_160,topic_161,topic_162,topic_163,topic_164,topic_165,topic_166,topic_167,topic_168,topic_169,topic_170,topic_171,topic_172,topic_173,topic_174,topic_175,topic_176,topic_177,topic_178,topic_179,topic_180,topic_181,topic_182,topic_183,topic_184,topic_185,topic_186,topic_187,topic_188,topic_189,topic_190,topic_191,topic_192,topic_193,topic_194,topic_195,topic_196,topic_197,topic_198,topic_199,topic_200,topic_201,topic_202,topic_203,topic_204,topic_205,topic_206,topic_207,topic_208,topic_209,topic_210,topic_211,topic_212,topic_213,topic_214,topic_215,topic_216,topic_217,topic_218,topic_219,topic_220,topic_221,topic_222,topic_223,topic_224,topic_225,topic_226,topic_227,topic_228,topic_229,topic_230,topic_231,topic_232,topic_233,topic_234,topic_235,topic_236,topic_237,topic_238,topic_239,topic_240,topic_241,topic_242,topic_243,topic_244,topic_245,topic_246,topic_247,topic_248,topic_249,topic_250,topic_251,topic_252,topic_253,topic_254,topic_255,topic_256,topic_257,topic_258,topic_259,topic_260,topic_261,topic_262,topic_263,topic_264,topic_265,topic_266,topic_267,topic_268,topic_269,topic_270,topic_271,topic_272,topic_273,topic_274,topic_275,topic_276,topic_277,topic_278,topic_279,topic_280,topic_281,topic_282,topic_283,topic_284,topic_285,topic_286,topic_287,topic_288,topic_289,topic_290,topic_291,topic_292,topic_293,topic_294,topic_295,topic_296,topic_297,topic_298,topic_299,topic_300,topic_301,topic_302,topic_303,topic_304,topic_305,topic_306,topic_307,topic_308,topic_309,topic_310,topic_311,topic_312,topic_313,topic_314,topic_315,topic_316,topic_317,topic_318,topic_319,topic_320,topic_321,topic_322,topic_323,topic_324,topic_325,topic_326,topic_327,topic_328,topic_329,topic_330,topic_331,topic_332,topic_333,topic_334,topic_335,topic_336,topic_337,topic_338,topic_339,topic_340,topic_341,topic_342,topic_343,topic_344,topic_345,topic_346,topic_347,topic_348,topic_349,topic_350,topic_351,topic_352,topic_353,topic_354,topic_355,topic_356,topic_357,topic_358,topic_359,topic_360,topic_361,topic_362,topic_363,topic_364,topic_365,topic_366,topic_367,topic_368,topic_369,topic_370,topic_371,topic_372,topic_373,topic_374,topic_375,topic_376,topic_377,topic_378,topic_379,topic_380,topic_381,topic_382,topic_383,topic_384,topic_385,topic_386,topic_387,topic_388,topic_389,topic_390,topic_391,topic_392,topic_393,topic_394,topic_395,topic_396,topic_397,topic_398,topic_399
0,0.179964,0.001757,-0.162870,-0.083024,-0.035702,-0.065108,-0.080027,-0.068378,0.047024,-0.020182,0.029596,-

In [93]:
topic_df.shape

(14337, 400)

In [95]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, r2_score, mean_squared_error, classification_report, make_scorer
from sklearn.metrics import average_precision_score

import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall

from pyspark.sql import SparkSession, Row
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import train_test_split

In [96]:
X = topic_df
y = df["fraud"]

In [98]:
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score, GridSearchCV


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                        random_state=4)

In [103]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11469, 400), (2868, 400), (11469,), (2868,))

In [104]:
'''Logistic Regression'''
logistic = LogisticRegression(penalty='l2',
                            dual=False,
                            tol=0.0001,
                            C=1.0,
                            fit_intercept=True,
                            intercept_scaling=1,
                            class_weight=None,
                            random_state=None,
                            solver='warn',
                            max_iter=100,
                            multi_class='warn',
                            verbose=0,
                            warm_start=False, n_jobs=None, l1_ratio=None)
logistic.fit(X_train, y_train)
logistic_pred = logistic.predict(X_test)
print("precision:", average_precision_score(y_test, logistic_pred), "\n F1:", f1_score(y_test, logistic_pred))


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


precision: 0.13052397356476483 
 F1: 0.06825938566552901


In [107]:
rf = RandomForestClassifier(n_estimators='warn',
                            criterion='gini',
                            max_depth=None,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features='auto',
                            max_leaf_nodes=None,
                            min_impurity_decrease=0.0,
                            min_impurity_split=None,
                            bootstrap=True,
                            oob_score=False,
                            n_jobs=None,
                            random_state=None,
                            verbose=0,
                            warm_start=False,
                            class_weight=None)
rf.fit(X_train, y_train)
importances = rf.feature_importances_
RF_pred = rf.predict(X_test)
print("precision:", average_precision_score(y_test, RF_pred), "\n F1:", f1_score(y_test, RF_pred))


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


precision: 0.1817504312960159 
 F1: 0.18927444794952683


In [110]:
svm = SVC(C=1.0,
            kernel='rbf',
            degree=3,
            gamma='auto_deprecated',
            coef0=0.0,
            shrinking=True,
            probability=False,
            tol=0.001,
            cache_size=200,
            class_weight=None,
            verbose=False,
            max_iter=-1,
            decision_function_shape='ovr',
            random_state=None)
svm.fit(X_train, y_train)
SVM_pred =  svm.predict(X_test)
print("precision:", average_precision_score(y_test, SVM_pred), "\n F1:", f1_score(y_test, SVM_pred))


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


precision: 0.09867503486750348 
 F1: 0.0


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [115]:
knn = KNeighborsClassifier(n_neighbors=5,
                            weights='uniform',
                            algorithm='auto',
                            leaf_size=30,
                            p=2,
                            metric='minkowski',
                            metric_params=None,
                            n_jobs=None)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
#output precision score
knn_precision = average_precision_score(y_test, knn_pred)
print("precision:", average_precision_score(y_test, knn_pred), "\n F1:", f1_score(y_test, knn_pred))


precision: 0.13273964916370384 
 F1: 0.1917098445595855


In [121]:
from tensorflow import keras
from tensorflow.keras.metrics import Precision, Recall

model_activation = keras.models.Sequential()

n_hidden = 25

model_activation.add(keras.layers.Dense(units=n_hidden,
                                        input_dim=1,
                                        activation='sigmoid'))
model_activation.add(keras.layers.Dense(units=1))
model_activation.add(keras.layers.Dropout(rate=0.5))

model_activation.compile(loss='mean_squared_error',
             optimizer='sgd',
             metrics=['Precision'])

In [126]:
model_activation.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 25)                50        
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 26        
_________________________________________________________________
dropout_4 (Dropout)          (None, 1)                 0         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_activation.fit(X_train, y_train,
                     epochs=10, batch_size=1, verbose = 0, validation_data=(X_test, y_test))

# Run models on final dataframe

In [131]:
df_final = pd.read_pickle('data/df_final.pickle')

In [151]:
df_final.head(10)

,body_length,channels,delivery_method,fb_published,has_logo,name_length,org_facebook,org_twitter,payee_name,sale_duration2,user_age,user_created,user_type,availability,has_state,has_country,has_address,has_venue_name,has_org_desc,has_org_name,payout_type_,payout_type_CHECK
0,3852,5,0.0,0,0,60,0.0,0.0,,33,36,1259613950,1,1.0,0,0,0,0,1,0,1,0
1,3561,8,0.0,0,1,90,0.0,0.0,,16,758,1259613950,1,1.0,1,0,0,1,1,1,0,0
2,533,8,0.0,0,1,88,17.0,14.0,,1,675,1259613950,1,1.0,0,0,0,0,0,0,1,0
3,630,8,0.0,0,1,67,0.0,0.0,,9,889,1259613950,1,1.0,0,0,0,0,1,0,1,0
4,990,5,0.0,0,1,89,0.0,0.0,,26,883,1259613950,1,1.0,0,0,0,0,1,0,0,0
5,1478,8,0.0,0,1,59,17.0,14.0,,10,732,1259613950,1,1.0,0,0,0,0,0,0,1,0
6,8836,6,0.0,0,1,188,0.0,0.0,,3,879,1259613950,1,1.0,1,1,0,1,0,0,1,0
7,649,8,0.0,0,1,171,0.0,0.0,,17,961,1259613950,1,1.0,1,1,0,0,0,0,1,0
8,593,8,0.0,0,1,165,17.0,14.0,,11,750,1259613950,1,1.0,0,0,0,0,0,0,1,0
9,45882,8,0.0,1,1,72,0.0,0.0,,66,0,1259613950,1,1.0,1,0,0,0,1,0,1,0


In [154]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 22 columns):
body_length          14337 non-null int64
channels             14337 non-null int64
delivery_method      14321 non-null float64
fb_published         14337 non-null int64
has_logo             14337 non-null int64
name_length          14337 non-null int64
org_facebook         14278 non-null float64
org_twitter          14278 non-null float64
payee_name           14337 non-null object
sale_duration2       14337 non-null int64
user_age             14337 non-null int64
user_created         14337 non-null int64
user_type            14337 non-null int64
availability         14337 non-null float64
has_state            14337 non-null int64
has_country          14337 non-null int64
has_address          14337 non-null int64
has_venue_name       14337 non-null int64
has_org_desc         14337 non-null int64
has_org_name         14337 non-null int64
payout_type_         14337 non-null uint8


In [165]:
X = df_final.drop(["payee_name", "delivery_method","org_facebook", "org_twitter"], axis=1)
y = df["fraud"]

In [159]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 18 columns):
body_length          14337 non-null int64
channels             14337 non-null int64
fb_published         14337 non-null int64
has_logo             14337 non-null int64
name_length          14337 non-null int64
sale_duration2       14337 non-null int64
user_age             14337 non-null int64
user_created         14337 non-null int64
user_type            14337 non-null int64
availability         14337 non-null float64
has_state            14337 non-null int64
has_country          14337 non-null int64
has_address          14337 non-null int64
has_venue_name       14337 non-null int64
has_org_desc         14337 non-null int64
has_org_name         14337 non-null int64
payout_type_         14337 non-null uint8
payout_type_CHECK    14337 non-null uint8
dtypes: float64(1), int64(15), uint8(2)
memory usage: 2.5 MB


In [156]:
X.shape, y.shape

((14337, 18), (14337,))

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                        random_state=4)

In [167]:
rf = RandomForestClassifier(n_estimators='warn',
                            criterion='gini',
                            max_depth=None,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features='auto',
                            max_leaf_nodes=None,
                            min_impurity_decrease=0.0,
                            min_impurity_split=None,
                            bootstrap=True,
                            oob_score=False,
                            n_jobs=None,
                            random_state=None,
                            verbose=0,
                            warm_start=False,
                            class_weight=None)
rf.fit(X_train, y_train)
importances = rf.feature_importances_
RF_pred = rf.predict(X_test)
print("precision:", average_precision_score(y_test, RF_pred), "\n F1:", f1_score(y_test, RF_pred))

precision: 0.09856193109294221 
 F1: 0.006711409395973155


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
